<a href="https://colab.research.google.com/github/wdempsey/AI4Health-Online-Experimentation/blob/main/part3_online.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Section 3: Synthetic HeartSteps and personalization


In [ ]:
## Import necessary 
import numpy as np
import scipy as sp
from sklearn.linear_model import LinearRegression

#Part 1: Overview on Contextual Bandits

- For each person in a study, let $t=1,\ldots, T$ denote a sequence of decision points.  
- At each decision time $t$,  we observe a state variable $S_t \in \mathbb{R}^p$.  
- After observing the state variable $S_t$, the _agent_ decides to take action $A_t \in \mathcal{A}$.  
- After observing state $S_t$ and taking action $A_t$, the agent receive a reward $R_t$ given by
$$
R_t = r(S_t, A_t) + \epsilon_t
$$
where $r(c,a)$ is a function that maps the state-action pair onto the real line and $\epsilon_t$ is a random error term, e.g., $\mathbb{E} [\epsilon_t] = 0$. 
- The triple (context, action, reward) at a sequence of decision points defines a _contextual bandit_ setting.  
- Here, the goal is to maximize the expected reward at every time point $\mathbb{E}[R_t \mid S_t, A_t=a] = r(S_t, a)$. 
- If we knew the reward function $r: \mathcal{S} \times \mathcal{S} \to \mathbb{R}$, then the optimal action given state $s$ is
$$
a^\star (s) = \max_{a \in \mathcal{A}} r(s, a)
$$

### Linear Contextual Bandit

- Assume that the reward structure follows
$$
r(s,a) = x(s,a)^\top \beta 
$$
where $x(s,a) \in \mathbb{R}^{p}$ is a $p$-dimensional summary of the state and $\beta \in \mathbb{R}^p$ is an unknown parameter.
- Before, we just wanted to build a good policy after collecting data on some participants.  
- Suppose now, we wish to minimize our __expected regret__ in making sub-optimal decisions on an individual
$$
\text{Regret}_t = \sum_{s=1}^t r(S_t, a_t^\star) - r(S_t, A_t)
$$

### Recall Susan's Talk!

- How does RL attempt to learn a policy that maximizes the reward (minimizes regret)?
- Two elements:
  - __Learning algorithm__: Used to incrementally update the model
  - __Action Selection__: Uses model to select next action
- Goal of action selection is to balance between __exploitation__ and __exploration__!

### Learning algorithm: Bayesian linear regression

- Here, we assume a common prior across participants but independence
- That is,
$$
R_{i, t} = \phi (S_{i,t}, A_{i,t})^\top w_i + \epsilon_{i,t}
$$
- And each $w_i$ is drawn with independent priors for each individual
- We take $\sigma^2$ to have an inverse-gamma prior and a normal prior for $w$ given $\sigma^2$ with mean $\mu_0$ and varaince $\sigma^2 \Lambda_0^{-1}$.
- __NOTE__: The prior distribution on $w$ can be chosen based on the prior MRT data!
- Using the standard Bayesian updates we have the posterior for $\sigma^2$ is an inverse-gamma with parameters $(a_t, b_t)$ and $w$ given $\sigma^2$ is normal with parameters $\mu_t$ and $\Lambda_t$ satisfying
\begin{align*}
\mu_t &= ( \phi(S_{1:t},A_{1:t})^\top \phi (S_{1:t},A_{1:t}) + \Lambda_0)^{-1}  (\Lambda_0 \mu_0 + \phi(S_{1:t},A_{1:t})^\top \phi (S_{1:t},A_{1:t}) \hat w) \\
\Lambda_t &= (\phi(S_{1:t},A_{1:t})^\top \phi (S_{1:t},A_{1:t}) + \Lambda_0) \\
a_t &= a_0 + t/2 \\
b_t &= b_0 + \frac{1}{2} (R_{1:t}^\top R_{1:t} + \mu_0^\top \Lambda_0 \mu_0 - \mu_n^\top \Lambda_n \mu_n)
\end{align*}


### Action Selection: Posterior sampling

- Sample $\sigma^2_t \sim InvGamma(a_t, b_t)$
- Sample $w_t \sim N(\mu_{t}, \sigma^2_t \Lambda_n)$ then
$$
A_t = \arg \max_{a \in \mathcal{A}} \phi(S_t, a)^\top w_i
$$

See [here](https://arxiv.org/pdf/2008.01571.pdf) for additional details on a slightly more complicated case.

In [ ]:
import collections

import glob

# Importing drive method from colab for accessing google drive
from google.colab import drive

 ## Importing Dataset from Google Drive

You can just go back and work directly with the MRT simulator based on Heartsteps V2 has been built in R and is available [here](https://drive.google.com/drive/folders/1rhCWugawTjEnwmagrOPwxNssrgIsnypT?usp=sharing)


In [ ]:
# Mounting drive
# This will require authentication : Follow the steps as guided
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Question 1: How would you use the warm-start policy 

## Question 2: What are some potential pitfalls in this approach?


## Exercise:  Code up a simple Thompson sampling algorithm for the next individual in the MRT.

## Part 1b: Online updating the Markov approach

- Suppose $\epsilon$-greedy action selection then update policy as follows:
- Suppose that the state-value function is parametrized according to $V(\pi, s; \theta^\pi) = \Phi (s)^\prime \theta$, then define 
$$
\Lambda_n (\pi, \theta^{\pi}) = \left[ n^{-1} \sum_{i=1}^n \sum_{t=1}^{T_i} \frac{\pi(A_{i,t}; S_{i,t})}{\hat \pi_n^{v-1}(A_{i,t}; S_{i,t})} \left( \gamma \Phi(S_{i,t}) \Phi(S_{i,t+1})^\prime - \Phi(S_{i,t}) \Phi(S_{i,t})^\prime\right) \right] \theta^\pi + n^{-1}  \sum_{i=1}^n \sum_{t=1}^{T_i} \left[\frac{\pi(A_{i,t}; S_{i,t})}{\hat \pi_n^{v-1}(A_{i,t}; S_{i,t})} R_{i,t} \Phi (S_{i,t}) \right]
$$
and we can estimate
$$
\hat \theta_n^\pi = \arg \min_{\theta^\pi \in \Theta} \left[ \Lambda_n (\pi, \theta^{\pi})^\prime \Lambda_n (\pi, \theta^{\pi}) + \lambda_n (\theta^{\pi})^\prime \theta^{\pi} \right]
$$
where $\lambda_n$ is a tuning parameter.
- See [here](https://arxiv.org/pdf/1611.03531v2.pdf) for additional details.

- __Question__: We could update every decision point.  What are some pitfalls of updating often?

# Additional topics not covered

- Constrained optimization under a randomized constraint
- Did we achieve personalization?
- Questions and discussion
